
# ViEWS 3 constituent models 
## ViEWS production system, cm level


This notebook trains a set of regression models for use in the monthly updated ViEWS predicting fatalities ensemble

The notebook does the following: 
1. Retrieves data through querysets and stores in DataSets, a list of dictionaries
2. Specifies the metadata of a number of models, stores in ModelList, a list of dictionaries
3. Trains the models in ModelList, stores the trained objects in model storage and prediction storage
4. Saves part of ModelList as csv and the rest as pickles


## Importing modules

In [1]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook
# sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression

from xgboost import XGBRegressor
from xgboost import XGBClassifier
from xgboost import XGBRFRegressor, XGBRFClassifier

from lightgbm import LGBMClassifier, LGBMRegressor

# Views 3
from viewser.operations import fetch
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage
from views_runs.storage import store, retrieve, fetch_metadata

from views_forecasts.extensions import *

# Other packages
import pickle as pkl

# Packages from Predicting Fatalies repository

#from HurdleRegression import * # Built on script from Geoff Hurdock: https://geoffruddock.com/building-a-hurdle-regression-estimator-in-scikit-learn/
#from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
from FetchData import FetchData, RetrieveFromList
from ViewsEstimators import *


/Users/havardhegre/anaconda3/envs/viewser/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Common parameters

In [2]:
#!conda list | grep views

In [3]:
# To do:
# find out why and where missingness occurs

In [4]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = 'Fatalities002'

# Generating a new run if necessary

#try:
#    ViewsMetadata().new_run(name=run_id,description='Developing the fatalities model for FCDO',min_month=1,max_month=999)
#except KeyError:
#    if 'devel' not in run_id:
#        warnings.warn('You are overwriting a production system')

RerunQuerysets = True

FutureStart = 508
steps = [*range(1, 36+1, 1)] # Which steps to train and predict for
fi_steps = [1,3,6,12,36] # Which steps to present feature importances for
#steps = [1,3,6,12,36]
#fi_steps = [1,3,6,12,36]

# Specifying partitions
calib_partitioner_dict = {"train":(121,396),"predict":(397,444)}
test_partitioner_dict = {"train":(121,444),"predict":(445,492)}
future_partitioner_dict = {"train":(121,492),"predict":(493,504)}
calib_partitioner =  views_runs.DataPartitioner({"calib":calib_partitioner_dict})
test_partitioner =  views_runs.DataPartitioner({"test":test_partitioner_dict})
future_partitioner =  views_runs.DataPartitioner({"future":future_partitioner_dict})

Mydropbox = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS'

# Retrieve data

In [5]:
# Specifying querysets
# Rerun if querysets have 
if RerunQuerysets:
    import cm_querysets

 .    hh_fat_cm_ged_ln_ultrashort; A dataset with 6 columns, with data between t 1 and 852. (213 units)
 .    hh_fatalities_ged_acled_ln; A dataset with 58 columns, with data between t 1 and 852. (213 units)
 .    fat_cm_conflict_history; A dataset with 29 columns, with data between t 1 and 852. (213 units)
 .    hh_fatalities_vdem_short; A dataset with 63 columns, with data between t 1 and 852. (213 units)
 .    hh_fatalities_wdi_short; A dataset with 32 columns, with data between t 1 and 852. (213 units)
 .    hh_topic_model; A dataset with 64 columns, with data between t 1 and 852. (213 units)
 .    hh_topic_model_short; A dataset with 39 columns, with data between t 1 and 852. (213 units)
 .    hh_all_features; A dataset with 181 columns, with data between t 1 and 852. (213 units)
 .    hh_broad; A dataset with 110 columns, with data between t 1 and 852. (213 units)
 .    hh_broad_short; A dataset with 76 columns, with data between t 1 and 852. (213 units)
 .    fatalities002_great

In [6]:
# Fetch data from the querysets
Datasets = FetchData(dev_id)

Fetching data using querysets; Fatalities002; returns as list of dictionaries containing datasets
 .    baseline: A dataset with 6 columns, with data between t = 1 and 852; 213 units.
 .    conflictlong_ln: A dataset with 58 columns, with data between t = 1 and 852; 213 units.
 .    conflict_ln: A dataset with 29 columns, with data between t = 1 and 852; 213 units.
 .    conflict_nolog: A dataset with 29 columns, with data between t = 1 and 852; 213 units.
 .    wdi_short: A dataset with 32 columns, with data between t = 1 and 852; 213 units.
 .    vdem_short: A dataset with 63 columns, with data between t = 1 and 852; 213 units.
 .    topics_short: A dataset with 39 columns, with data between t = 1 and 852; 213 units.
 .    broad: A dataset with 110 columns, with data between t = 1 and 852; 213 units.
 .    gh: A dataset with 57 columns, with data between t = 1 and 852; 213 units.
 .    hh20: A dataset with 39 columns, with data between t = 1 and 852; 213 units.
 .    all_features: A 

# Generating predictions
Using the new partitioning/stepshifting syntax. Training models for A: calibration partition and B: test partition, to test out some calibration routines. Most models trained with ln_ged_sb_best as outcome, but also one model with ged_sb_best to see whether that improves calibration on its own.

In [7]:
dev_id

'Fatalities002'

In [8]:
Datasets[0]['df'].head()

ln_ged_sb_dep  ln_ged_sb  wdi_sp_pop_totl  \
month_id country_id                                              
1        1                     0.0        0.0         780153.0   
         2                     0.0        0.0         359531.0   
         3                     0.0        0.0        1084744.0   
         4                     0.0        0.0       15182611.0   
         5                     0.0        0.0         155525.0   

                     decay_ged_sb_5  decay_ged_os_5  splag_1_decay_ged_sb_5  
month_id country_id                                                          
1        1                      0.0             0.0                     0.0  
         2                      0.0             0.0                     0.0  
         3                      0.0             0.0                     0.0  
         4                      0.0             0.0                     0.0  
         5                      0.0             0.0                     0.0

In [9]:
from views_runs import ModelMetadata 
help(ModelMetadata)

Help on class ModelMetadata in module views_schema.models:

class ModelMetadata(pydantic.main.BaseModel)
 |  ModelMetadata(*, author: str, queryset_name: str, train_start: int, train_end: int, steps: List[int] = None, training_date: datetime.datetime) -> None
 |  
 |  ModelMetadata
 |  =============
 |  
 |  Data used to organize model objects.
 |  
 |  parameters:
 |      author (str): Name of the user that authored the model object.
 |      queryset_name (str): Name of the queryset used to train the model
 |      train_start (int): Month identifier for training start date
 |      train_start (int): Month identifier for training end date
 |      training_date (datetime.datetime): Timestamp for training date (use datetime.datetime.now())
 |  
 |  example:
 |  
 |      # Instantiate the class with values
 |  
 |      my_metadata = ModelMetadata(
 |          author = "my_name",
 |          queryset_name = "my_queryset",
 |          train_start = 1,
 |          train_end = 300,
 |        

## Checking missingness and infinity values

In [10]:
N=51
df = Datasets[0]['df']
for col in df.iloc[: , :N].columns:
    print(col,len(df[col]), 'missing:', df[col].isnull().sum(), 'infinity:', np.isinf(df).values.sum())


ln_ged_sb_dep 158230 missing: 0 infinity: 0
ln_ged_sb 158230 missing: 0 infinity: 0
wdi_sp_pop_totl 158230 missing: 11 infinity: 0
decay_ged_sb_5 158230 missing: 0 infinity: 0
decay_ged_os_5 158230 missing: 0 infinity: 0
splag_1_decay_ged_sb_5 158230 missing: 0 infinity: 0


## Identify early stopping parameter

See the Early_stopping_experiment notebook for how we arrived at the early stopping parameters for the XGBoost models.

# Specify models in ensemble

In [11]:
# The ModelList is a list of dictionaries that define a range of models for the project

ModelList = []
nj=12

model = {
    'modelname':     'fat_baseline_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'baseline',
    'queryset':      'hh_fatalities_ged_ln_ultrashort',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

# Model: GED logged dependent variable, logged conflict history variables, gradient boosting
model = {
    'modelname': 'fat_conflicthistory_gbm',
    'algorithm': GradientBoostingRegressor(), 
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)       
    

model = {
    'modelname': 'fat_conflicthistory_hurdle_lgb',
    'algorithm': HurdleRegression(clf_name = 'LGBMClassifier', reg_name = 'LGBMRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflict_ln',
    'queryset': "fat_cm_conflict_history",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_conflicthistory_long_xgb',
    'algorithm': XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'conflictlong_ln',
    'queryset': "hh_fatalities_ged_acled_ln",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_vdem_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'vdem_short',
    'queryset':  "hh_fatalities_vdem_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_wdi_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'wdi_short',
    'queryset':  "hh_fatalities_wdi_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_rf',
    'algorithm': XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics_short',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':  'fat_topics_histgbm',
    'algorithm': HistGradientBoostingRegressor(max_iter=200),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'topics_short',
    'queryset':   "hh_topic_model_short",
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname':  'fat_broad_xgb',
    'algorithm':  XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'broad',
    'queryset':   'hh_broad',
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname':  'fatalities002_greatest_hits_hurdle_xgb',
    'algorithm':  HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar':     "ln_ged_sb_dep",
    'data_train':    'gh',
    'queryset':   'fatalities002_greatest_hits',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_hurdle_rf',
    'algorithm': HurdleRegression(clf_name = 'RFClassifier', reg_name = 'RFRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname': 'fat_hh20_hurdle_xgb',
    'algorithm': HurdleRegression(clf_name = 'XGBClassifier', reg_name = 'XGBRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)


model = {
    'modelname': 'fat_hh20_hurdle_lgb',
    'algorithm': HurdleRegression(clf_name = 'LGBMClassifier', reg_name = 'LGBMRegressor'),
    'depvar': "ln_ged_sb_dep",
    'data_train':    'hh20',
    'queryset': 'hh_20_features',
    'preprocessing': 'float_it',
}
ModelList.append(model)


# PCA models: need to implement a PCA preprocessing function first.
model = {
    'modelname':      'fat_all_pca3_xgb',
    'algorithm':      XGBRegressor(n_estimators=100,learning_rate=0.05,n_jobs=nj),
    'depvar':         "ln_ged_sb_dep",
    'data_train':     'pca_all',
    'queryset':      'hh_all_features',
    'preprocessing': 'pca_it',
}
ModelList.append(model)


model = {
    'modelname':     'fatalities002_aquastat_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'aquastat',
    'queryset':      'Fatalities002_aquastat',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':     'fatalities002_faostat_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'faostat',
    'queryset':      'Fatalities002_faostat',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':     'fatalities002_faoprices_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'faoprices',
    'queryset':      'Fatalities002_faoprices',
    'preprocessing': 'float_it',
}
ModelList.append(model)

model = {
    'modelname':     'fatalities002_imfweo_rf',
    'algorithm':     XGBRFRegressor(n_estimators=300,n_jobs=nj),
    'depvar':        "ln_ged_sb_dep",
    'data_train':    'imfweo',
    'queryset':      'Fatalities001_imfweo',
    'preprocessing': 'float_it',
}
ModelList.append(model)


i = 0
for model in ModelList:
    print(i, model['modelname'], model['data_train'])
    i = i + 1

0 fat_baseline_rf baseline
1 fat_conflicthistory_rf conflict_ln
2 fat_conflicthistory_gbm conflict_ln
3 fat_conflicthistory_hurdle_lgb conflict_ln
4 fat_conflicthistory_long_xgb conflictlong_ln
5 fat_vdem_hurdle_xgb vdem_short
6 fat_wdi_rf wdi_short
7 fat_topics_rf topics_short
8 fat_topics_histgbm topics_short
9 fat_broad_xgb broad
10 fatalities002_greatest_hits_hurdle_xgb gh
11 fat_hh20_hurdle_rf hh20
12 fat_hh20_hurdle_xgb hh20
13 fat_hh20_hurdle_lgb hh20
14 fat_all_pca3_xgb pca_all
15 fatalities002_aquastat_rf aquastat
16 fatalities002_faostat_rf faostat
17 fatalities002_faoprices_rf faoprices
18 fatalities002_imfweo_rf imfweo


In [12]:
# Loop that checks whether the model exists, retrains if not, 
# and stores the predictions if they have not been stored before for this run.
# To do: set the data_preprocessing to the function in the model dictionary

level = 'cm'
includeFuture = False

from views_runs import Storage, StepshiftedModels
from views_partitioning.data_partitioner import DataPartitioner
from viewser import Queryset, Column
from views_runs import operations
from views_runs.run_result import RunResult

i = 0
for model in ModelList[0:1]:
    force_retrain = False
    modelstore = storage.Storage()
    ct = datetime.now()
    print(i, model['modelname'])
    print('Calibration partition', ct)
    model['Algorithm_text'] = str(model['algorithm'])
    model['RunResult_calib'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"calib":calib_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "calib",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_calib',
            author_name        = "HH",
    )

    model['predstore_calib'] = level +  '_' + model['modelname'] + '_calib'
    ct = datetime.now()
    print('Trying to retrieve predictions', ct)
    try:
        predictions_calib = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_calib'])
    except KeyError:
        print(model['predstore_calib'], ', run',  run_id, 'does not exist, predicting')
        predictions_calib = model['RunResult_calib'].run.predict("calib","predict", model['RunResult_calib'].data)
        predictions_calib.forecasts.set_run(run_id)
        predictions_calib.forecasts.to_store(name=model['predstore_calib'])

    ct = datetime.now()
    print('Test partition', ct)
    modelstore = storage.Storage()
    model['RunResult_test'] = RunResult.retrain_or_retrieve(
            retrain            = force_retrain,
            store              = modelstore,
            partitioner        = DataPartitioner({"test":test_partitioner_dict}),
            stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
            dataset            = RetrieveFromList(Datasets,model['data_train']),
            queryset_name      = model['queryset'],
            partition_name     = "test",
            timespan_name      = "train",
            storage_name       = model['modelname'] + '_test',
            author_name        = "HH",
    )
    ct = datetime.now()
    print('Trying to retrieve predictions', ct)
    model['predstore_test'] = level +  '_' + model['modelname'] + '_test'
    try:
        predictions_test = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_test'])
    except KeyError:
        print(model['predstore_test'], ', run', run_id, 'does not exist, predicting')
        predictions_test = model['RunResult_test'].run.predict("test","predict",model['RunResult_test'].data)
        predictions_test.forecasts.set_run(run_id)
        predictions_test.forecasts.to_store(name=model['predstore_test'])
    # Predictions for true future
    if includeFuture:
        ct = datetime.now()
        print('Future', ct)
        modelstore = storage.Storage()
        model['RunResult_future'] = RunResult.retrain_or_retrieve(
                retrain            = force_retrain,
                store              = modelstore,
                partitioner        = DataPartitioner({"test":future_partitioner_dict}),
                stepshifted_models = StepshiftedModels(model['algorithm'], steps, model['depvar']),
                dataset            = RetrieveFromList(Datasets,model['data_train']),
                queryset_name      = model['queryset'],
                partition_name     = "test",
                timespan_name      = "train",
                storage_name       = model['modelname'] + '_future',
                author_name        = "HH",
        )
        ct = datetime.now()
        print('Trying to retrieve predictions', ct)
        model['predstore_future'] = level +  '_' + model['modelname'] + '_f' + str(FutureStart)
        try:
            predictions_future = pd.DataFrame.forecasts.read_store(run=run_id, name=model['predstore_future'])
        except KeyError:
            print(model['predstore_future'], ', run', run_id, 'does not exist, predicting')
            predictions_future = model['RunResult_future'].run.future_point_predict(FutureStart,model['RunResult_future'].data)
            predictions_future.forecasts.set_run(run_id)
            predictions_future.forecasts.to_store(name=model['predstore_future'])  
    print('**************************************************************')
    model['algorithm'] = []
    i = i + 1

print('All done')

0 fat_baseline_rf
Calibration partition 2022-06-21 16:45:49.816201
 * == Performing a run: "fat_baseline_rf_calib" == * 
Model object named "fat_baseline_rf_calib" with equivalent metadata already exists.
Fetching "fat_baseline_rf_calib" from storage
Trying to retrieve predictions 2022-06-21 16:46:34.853778
pr_46_cm_fat_baseline_rf_calib.parquet
Test partition 2022-06-21 16:46:40.231799
 * == Performing a run: "fat_baseline_rf_test" == * 
Model object named "fat_baseline_rf_test" with equivalent metadata already exists.
Fetching "fat_baseline_rf_test" from storage
Trying to retrieve predictions 2022-06-21 16:47:26.327661
pr_46_cm_fat_baseline_rf_test.parquet
**************************************************************
All done


In [ ]:
# Exploring the future predictions


predictions_test.xs(246,level=1).tail()

## Notes on training time for the various algorithms:

In [ ]:
#These are calculated in minutes for the hh20 feature set (with about 40 features), for all 36 steps, calibration (c) and test (t) partitions, also include generating predictions, and are approximate:

#nj=12 (number of threads)
#scikit random forest:        21:13 (c), 26:20 (t) RandomForestRegressor(n_estimators=200, n_jobs=nj)
#XGB random forest:           06:02 (c), 07:51 (t) XGBRFRegressor(n_estimators=300,n_jobs=nj)
#scikit gbm:                  13:59 (c), 15:55 (t) GradientBoostingRegressor(), 
#scikit hurdle random forest: 07:32 (c), 09:49 (t) For both clf and reg: (n_estimators=200, n_jobs=nj)
#XGB hurdle xgb:              01:26 (c), 01:32 (t) For both clf and reg:                n_estimators=200,tree_method='hist',n_jobs=nj)
#scikit histgbm:              01:17 (c), 01:20 (t) HistGradientBoostingRegressor(max_iter=200)
#XGB xgb:                     01:00 (c), 01:04 (t) XGBRegressor(n_estimators=200,tree_method='hist',n_jobs=nj)
#lightgbm gbm:                00:25 (c), --    (t) LGBMRegressor(n_estimators=100,num_threads=8)

# Various helper functions and tools....

In [ ]:
!conda list | grep views-forecasts

# Retrieving external forecasts

In [18]:
# Retrieve David's Markov models
# To do: rewrite the model dictionary to the new, slimmer version.
DRList = []


model = {
    'modelname':   'fat_hh20_Markov_glm',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
ModelList.append(model)
DRList.append(model)

model = {
    'modelname':   'fat_hh20_Markov_rf',
    'algorithm': [],
    'depvar': "ln_ged_sb_dep",
    'data_train':      'hh20',
    'queryset': 'hh_20_features',
}
ModelList.append(model)
DRList.append(model)



In [19]:
path = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/cm/preds/'

DRList[0]['predictions_file_calib'] = path + 'vmm_glm_hh20_0125_alt_calib.csv'
DRList[0]['predictions_file_test'] = path + 'vmm_glm_hh20_0125_alt_test.csv'
DRList[0]['predictions_file_future'] = path + 'vmm_glm_hh20_506.csv'

DRList[1]['predictions_file_calib'] = path + 'vmm_rf_hh20_0125_alt_calib.csv'
DRList[1]['predictions_file_test'] = path + 'vmm_rf_hh20_0125_alt_test.csv'
DRList[1]['predictions_file_future'] = path + 'vmm_rf_hh20_505.csv'

In [24]:

for model in ModelList:
    print(model['modelname'])

fat_baseline_rf
fat_conflicthistory_rf
fat_conflicthistory_gbm
fat_conflicthistory_hurdle_lgb
fat_conflicthistory_long_xgb
fat_vdem_hurdle_xgb
fat_wdi_rf
fat_topics_rf
fat_topics_histgbm
fat_broad_xgb
fatalities002_greatest_hits_hurdle_xgb
fat_hh20_hurdle_rf
fat_hh20_hurdle_xgb
fat_hh20_hurdle_lgb
fat_all_pca3_xgb
fatalities002_aquastat_rf
fatalities002_faostat_rf
fatalities002_faoprices_rf
fatalities002_imfweo_rf
fat_hh20_Markov_glm
fat_hh20_Markov_rf


In [15]:
# Storing Markov models in central storage
# Retrieving dependent variable
target_calib = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_calib', run=run_id)['ln_ged_sb_dep']
target_test = pd.DataFrame.forecasts.read_store('cm_fat_conflicthistory_rf_test', run=run_id)['ln_ged_sb_dep']
level = 'cm'
for model in DRList:
    df_calib = pd.read_csv(model['predictions_file_calib'],index_col=['month_id','country_id'])
    df_test = pd.read_csv(model['predictions_file_test'],index_col=['month_id','country_id'])
    df_future = pd.read_csv(model['predictions_file_future'],index_col=['month_id','country_id'])
    df_calib['ln_ged_sb_dep'] = target_calib
    df_test['ln_ged_sb_dep'] = target_test
    df_future['ln_ged_sb_dep'] = np.nan # Empty dependent variable column for consistency/required by prediction storage function
    stored_modelname = level + '_' + model['modelname'] + '_calib'
    df_calib.forecasts.set_run(run_id)
    df_calib.forecasts.to_store(name=stored_modelname, overwrite=True)
    stored_modelname = level + '_' + model['modelname'] + '_test'
    df_test.forecasts.set_run(run_id)
    df_test.forecasts.to_store(name=stored_modelname, overwrite=True)    

pr_46_cm_fat_conflicthistory_rf_calib.parquet
pr_46_cm_fat_conflicthistory_rf_test.parquet


In [25]:
# Removing run result objects before saving
# These should perhaps be discarded immediately above
for model in ModelList[0:-3]:
    model['RunResult_calib'] = []
    model['RunResult_test'] = []

In [17]:
ModelList

[{'modelname': 'fat_baseline_rf',
  'algorithm': [],
  'depvar': 'ln_ged_sb_dep',
  'data_train': 'baseline',
  'queryset': 'hh_fatalities_ged_ln_ultrashort',
  'preprocessing': 'float_it',
  'Algorithm_text': "XGBRFRegressor(base_score=None, booster=None, colsample_bylevel=None,\n               colsample_bytree=None, enable_categorical=False, gamma=None,\n               gpu_id=None, importance_type=None, interaction_constraints=None,\n               max_delta_step=None, max_depth=None, min_child_weight=None,\n               missing=nan, monotone_constraints=None, n_estimators=300,\n               n_jobs=12, num_parallel_tree=None, objective='reg:squarederror',\n               predictor=None, random_state=None, reg_alpha=None,\n               scale_pos_weight=None, tree_method=None,\n               validate_parameters=None, verbosity=None)",
  'RunResult_calib': [],
  'predstore_calib': 'cm_fat_baseline_rf_calib',
  'RunResult_test': [],
  'predstore_test': 'cm_fat_baseline_rf_test'},


# Save the model list of dictionaries

In [26]:
ModelList_df = pd.DataFrame.from_dict(ModelList)
localpath = '/Users/havardhegre/Dropbox (ViEWS)/ViEWS/Projects/PredictingFatalities/Predictions/'

filename = localpath + 'Model_cm_' + model['modelname'] + '_'+ dev_id + '.csv'
ModelList_df.to_csv(filename)
gitname = 'ModelList_cm_wide_' + dev_id + '.csv'
ModelList_df.to_csv(gitname)